# Parsing Data

In [1]:
import os
import tifffile
import numpy as np
import torch
from matplotlib import pyplot as plt

In [2]:
__BASE_DIR__ = "/Users/lrob/Desktop/4Y/VFFP/"


mcs_dir = __BASE_DIR__+"mcx"
dataset_dirs = {}
def map_dir(dir: str, dir_dict: dict):
    for item in os.listdir(dir):
        if not item.startswith('.'):
            dir_dict[item]=(dir+"/"+item)

map_dir(mcs_dir, dataset_dirs)
print(dataset_dirs)
samples = list(dataset_dirs.keys())
print(samples)
torch.has_mps


{'12_baseline_norm.tif': '/Users/lrob/Desktop/4Y/VFFP/mcx/12_baseline_norm.tif', '08_baseline_norm.tif': '/Users/lrob/Desktop/4Y/VFFP/mcx/08_baseline_norm.tif', '11_baseline_norm.tif': '/Users/lrob/Desktop/4Y/VFFP/mcx/11_baseline_norm.tif', '09_baseline_norm.tif': '/Users/lrob/Desktop/4Y/VFFP/mcx/09_baseline_norm.tif', '10_baseline_norm.tif': '/Users/lrob/Desktop/4Y/VFFP/mcx/10_baseline_norm.tif', '07_baseline_norm.tif': '/Users/lrob/Desktop/4Y/VFFP/mcx/07_baseline_norm.tif'}
['12_baseline_norm.tif', '08_baseline_norm.tif', '11_baseline_norm.tif', '09_baseline_norm.tif', '10_baseline_norm.tif', '07_baseline_norm.tif']


True

In [25]:
from skimage.util import view_as_blocks

def clip_sample(f, dims):
    h,w = dims
    split = np.squeeze(view_as_blocks(f, (f.shape[0], h, w)))
    n = split.shape[0]*split.shape[1]
    split = np.reshape(split, (n * f.shape[0], h, w))
    return split

def pad_sample(f, dims):
    h,w = dims
    pad_h = h - f.shape[1]%h if (f.shape[1]%h != 0) else 0
    pad_w = w - f.shape[2]%w if (f.shape[2]%w != 0) else 0
    
    pad_1 = pad_h//2
    pad_2 = pad_h - pad_1
    pad_3 = pad_w//2
    pad_4 = pad_w - pad_3
    
    f_pad = np.pad(f, ((0,0), (pad_1,pad_2), (pad_3, pad_4)), 'constant')
    return f_pad

def crop_sample(f, d):
    crop_h_s = (f.shape[1]-d)//2
    crop_w_s = (f.shape[2]-d)//2
    crop_h_e = (f.shape[1])-crop_h_s
    crop_w_e = (f.shape[2])-crop_w_s

    return f[:,crop_h_s:crop_h_e,crop_w_s:crop_w_e]

def generate_toyMCS(sample, timeCrop_dim = None, spaceCrop = False, spaceCrop_dim = None):
        print("Selected sample: ")
        print(sample)

        sample_np = tifffile.memmap(dataset_dirs[sample], mode='r')
        sample_np = crop_sample(sample_np, 140)

        if timeCrop_dim is None:
            timeCrop_dim = sample_np.shape[0]

        #if spaceCrop_dim is None:
            #spaceCrop_dim = (sample_np.shape[-1],sample_np.shape[-1])

        #sample_np = pad_sample(sample_np, spaceCrop_dim)
        num_subclips = sample_np.shape[0]//timeCrop_dim

        #if spaceCrop:
        #        if spaceCrop_dim is None:
        #                raise ValueError('Unspecified spaceCrop_dim (eg. (64,64) ).')
        #        sample_np = clip_sample(sample_np, spaceCrop_dim)

        dataset = np.expand_dims(sample_np, -3)
        clips = np.empty((2,num_subclips,2), dtype=np.uint64)

        for subclip_idx in range(num_subclips):
                clips[0][subclip_idx][0] = subclip_idx * timeCrop_dim
                clips[0][subclip_idx][1] = int(timeCrop_dim//2)
                clips[1][subclip_idx][1] = int(timeCrop_dim//2)
                clips[1][subclip_idx][0] = clips[0][subclip_idx][0]+clips[0][subclip_idx][1]+1
        
        print("dataset.shape = "+str(dataset.shape))
        print("clips.shape = "+str(clips.shape))
        print()
        return dataset, clips

In [26]:
d_train, c_train = generate_toyMCS(samples[0], timeCrop_dim=100)

d_test, c_test = generate_toyMCS(samples[0], timeCrop_dim=100)

d_val, c_val = generate_toyMCS(samples[0], timeCrop_dim=100)



Selected sample: 
12_baseline_norm.tif
dataset.shape = (29000, 1, 140, 140)
clips.shape = (2, 290, 2)

Selected sample: 
12_baseline_norm.tif
dataset.shape = (29000, 1, 140, 140)
clips.shape = (2, 290, 2)

Selected sample: 
12_baseline_norm.tif
dataset.shape = (29000, 1, 140, 140)
clips.shape = (2, 290, 2)



In [21]:
np.savez(__BASE_DIR__+"VFFP/ToyMCS/nxn/toy-MCS-train.npz", input_raw_data = d_train, clips = c_train)
np.savez(__BASE_DIR__+"VFFP/ToyMCS/nxn/toy-MCS-test.npz", input_raw_data = d_test, clips = c_test)
np.savez(__BASE_DIR__+"VFFP/ToyMCS/nxn/toy-MCS-val.npz", input_raw_data = d_val, clips = c_val)

In [22]:
print(np.std(d_train))
print(np.std(d_test))
print(np.std(d_val))

print(np.var(d_train))
print(np.var(d_test))
print(np.var(d_val))

0.5318002
0.5318002
0.5318002


In [5]:
with tifffile.TiffWriter('temp.tif') as tif:
    for frame in sample_np_split[0]:
        tif.write(frame, contiguous=True)

NameError: name 'sample_np_split' is not defined